In [ ]:
from circuit_functions import *
from qiskit import QuantumCircuit, transpile, Aer, IBMQ, execute
import numpy as np
from numpy import pi
from time import sleep

## Plays a note from each column of the circuit

In [ ]:
"""Insert circuit in this cell"""

qasm = """
OPENQASM 2.0;
include "qelib1.inc";

qreg q[3];
creg c[3];

h q[0];
h q[1];
s q[0];
s q[1];
t q[0];
s q[1];
tdg q[0];
cx q[1],q[2];
t q[0];
s q[1];
h q[1];
cx q[0],q[1];
t q[0];
h q[1];
t q[0];
"""

circuit = QuantumCircuit.from_qasm_str(qasm)
circuit.draw()
print(f'{len(circuit.qubits)} qubits')

In [ ]:
# Get circuits by column
sub_circuits = get_circuits_by_column(circuit)

In [ ]:
# Use first column as the starting state vector
state_vector = get_state_vector(sub_circuits[0])
print_vector(state_vector)

sub_circuits[0].draw()
print_vector(get_phases(state_vector), comment='phases')
print_vector(get_probabilities(state_vector), comment='probabilities')

In [ ]:
# Iterate through circuit
prev_state_vector = get_state_vector(sub_circuits[0])
for i in range(1, len(sub_circuits)):
    print(f'Column {i}')
    sub_circuit = sub_circuits[i]
    unitary_matrix = get_unitary_matrix(sub_circuit)
    
    # Multiply this column's unitary matrix with previous state_vector
    state_vector = np.matmul(unitary_matrix, prev_state_vector)
    print_vector(state_vector, comment='state')
    print_vector(get_phases(state_vector), comment='phases')
    print_vector(get_probabilities(state_vector), comment='probabilities')
    
    notes = get_notes(state_vector)
    print('notes', notes)
    
    play_notes(notes)
    sleep(1) # Add some delay
        
    # Update this state vector to next iteration's prev_state_vector
    prev_state_vector = state_vector
        
    print('\n')